In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2024-11-25 00:01:52,999] A new study created in memory with name: no-name-a8b26ddd-2470-4b74-b713-585533613637
[I 2024-11-25 00:01:53,422] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 76, 'max_depth': 13}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-25 00:01:54,222] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 184, 'max_depth': 14}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-25 00:01:55,016] Trial 2 finished with value: 0.7765363128491619 and parameters: {'n_estimators': 185, 'max_depth': 16}. Best is trial 2 with value: 0.7765363128491619.
[I 2024-11-25 00:01:55,585] Trial 3 finished with value: 0.7541899441340782 and parameters: {'n_estimators': 154, 'max_depth': 3}. Best is trial 2 with value: 0.7765363128491619.
[I 2024-11-25 00:01:56,182] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 134, 'max_depth': 6}. Best is trial 2 with value: 0.7765363

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 70, 'max_depth': 17}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

### Random Sampler

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-11-25 00:04:20,460] A new study created in memory with name: no-name-4d2e0915-aad5-4221-a4a5-404eb3516cd4
[I 2024-11-25 00:04:21,144] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 164, 'max_depth': 6}. Best is trial 0 with value: 0.7635009310986964.
[I 2024-11-25 00:04:21,621] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 121, 'max_depth': 5}. Best is trial 1 with value: 0.7709497206703911.
[I 2024-11-25 00:04:22,295] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 134, 'max_depth': 17}. Best is trial 2 with value: 0.7746741154562384.
[I 2024-11-25 00:04:22,736] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 98, 'max_depth': 4}. Best is trial 2 with value: 0.7746741154562384.
[I 2024-11-25 00:04:23,036] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 64, 'max_depth': 18}. Best is trial 2 with value: 0.774674115

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 19}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


### Grid Sampler

In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-11-25 00:05:37,802] A new study created in memory with name: no-name-910e66a0-d8f8-440b-9790-804a7a603a7d


[I 2024-11-25 00:05:38,259] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-25 00:05:38,904] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-25 00:05:39,125] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-11-25 00:05:39,571] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-11-25 00:05:40,026] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-11-25 00:05:40,257] Trial 5 finished with value: 0.7579143389199254 and parameters: {'n_estima

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [ ]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [16]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [17]:
# 1. Optimization History
plot_optimization_history(study).show()

In [18]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [19]:
# 3. Slice Plot
plot_slice(study).show()

In [20]:
# 4. Contour Plot
plot_contour(study).show()

In [21]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [22]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [23]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-11-25 00:16:27,541] A new study created in memory with name: no-name-2aad46a7-ea20-40c9-b153-9b9165a4b703
[I 2024-11-25 00:16:27,807] Trial 0 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 71, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 10, 'bootstrap': False}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-11-25 00:16:30,587] Trial 1 finished with value: 0.7709497206703911 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 207, 'learning_rate': 0.01342656799124919, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.7709497206703911.
[I 2024-11-25 00:16:31,165] Trial 2 finished with value: 0.7746741154562384 and parameters: {'classifier': 'RandomForest', 'n_estimators': 151, 'max_depth': 13, 'min_samples_split': 6, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 2 with value: 0.7746741154562384.
[I 2024-11-25 00:16:32,062] Trial 

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13964790337895397, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.770950,2024-11-25 00:16:27.543855,2024-11-25 00:16:27.806891,0 days 00:00:00.263036,NaN,False,RandomForest,NaN,NaN,NaN,6.0,10.0,7.0,71.0,COMPLETE
1,1,0.770950,2024-11-25 00:16:27.808855,2024-11-25 00:16:30.587191,0 days 00:00:02.778336,NaN,NaN,GradientBoosting,NaN,NaN,0.013427,10.0,7.0,5.0,207.0,COMPLETE
2,2,0.774674,2024-11-25 00:16:30.589196,2024-11-25 00:16:31.164833,0 days 00:00:00.575637,NaN,False,RandomForest,NaN,NaN,NaN,13.0,8.0,6.0,151.0,COMPLETE
3,3,0.759777,2024-11-25 00:16:31.166795,2024-11-25 00:16:32.062248,0 days 00:00:00.895453,NaN,True,RandomForest,NaN,NaN,NaN,11.0,8.0,7.0,213.0,COMPLETE
4,4,0.785847,2024-11-25 00:16:32.063108,2024-11-25 00:16:32.204764,0 days 00:00:00.141656,28.030209,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.783985,2024-11-25 00:16:56.324805,2024-11-25 00:16:56.350701,0 days 00:00:00.025896,1.065821,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.765363,2024-11-25 00:16:56.351742,2024-11-25 00:16:56.387830,0 days 00:00:00.036088,0.309485,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.711359,2024-11-25 00:16:56.388829,2024-11-25 00:16:56.412739,0 days 00:00:00.023910,0.130672,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2024-11-25 00:16:56.413730,2024-11-25 00:16:56.435136,0 days 00:00:00.021406,0.232363,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

SVM                 74
RandomForest        16
GradientBoosting    10
Name: params_classifier, dtype: int64

In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.742644
RandomForest        0.769669
SVM                 0.773114
Name: value, dtype: float64

In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2024-11-25 11:24:30,398] A new study created in memory with name: no-name-44968a4d-f009-4b5b-82fd-6c7564f67399


[0]	train-mlogloss:0.84756	eval-mlogloss:0.83984
[1]	train-mlogloss:0.67815	eval-mlogloss:0.66032
[2]	train-mlogloss:0.55321	eval-mlogloss:0.52929
[3]	train-mlogloss:0.46070	eval-mlogloss:0.43272
[4]	train-mlogloss:0.38703	eval-mlogloss:0.35567
[5]	train-mlogloss:0.32761	eval-mlogloss:0.28972
[6]	train-mlogloss:0.28233	eval-mlogloss:0.24174
[7]	train-mlogloss:0.24520	eval-mlogloss:0.20416
[8]	train-mlogloss:0.21640	eval-mlogloss:0.17436
[9]	train-mlogloss:0.19499	eval-mlogloss:0.15091
[10]	train-mlogloss:0.17662	eval-mlogloss:0.12797
[11]	train-mlogloss:0.16296	eval-mlogloss:0.11234
[12]	train-mlogloss:0.15132	eval-mlogloss:0.09628
[13]	train-mlogloss:0.14271	eval-mlogloss:0.08640
[14]	train-mlogloss:0.13996	eval-mlogloss:0.08422
[15]	train-mlogloss:0.13694	eval-mlogloss:0.07915
[16]	train-mlogloss:0.13483	eval-mlogloss:0.07507
[17]	train-mlogloss:0.13261	eval-mlogloss:0.07201
[18]	train-mlogloss:0.13062	eval-mlogloss:0.06808
[19]	train-mlogloss:0.12922	eval-mlogloss:0.06584
[20]	train

[I 2024-11-25 11:24:33,130] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.1862281922948573, 'alpha': 0.0002686829395108523, 'eta': 0.2010747597313131, 'gamma': 1.8671245320624107e-07, 'max_depth': 9, 'min_child_weight': 4, 'subsample': 0.7537517770115805, 'colsample_bytree': 0.9625969913585541}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.74584	eval-mlogloss:0.72633
[1]	train-mlogloss:0.57332	eval-mlogloss:0.54083
[2]	train-mlogloss:0.43913	eval-mlogloss:0.39752
[3]	train-mlogloss:0.35377	eval-mlogloss:0.30434
[4]	train-mlogloss:0.30453	eval-mlogloss:0.24929
[5]	train-mlogloss:0.26491	eval-mlogloss:0.20352
[6]	train-mlogloss:0.26327	eval-mlogloss:0.19884
[7]	train-mlogloss:0.26330	eval-mlogloss:0.19895
[8]	train-mlogloss:0.24846	eval-mlogloss:0.18189
[9]	train-mlogloss:0.23865	eval-mlogloss:0.17006
[10]	train-mlogloss:0.23877	eval-mlogloss:0.17043
[11]	train-mlogloss:0.23875	eval-mlogloss:0.17032
[12]	train-mlogloss:0.23783	eval-mlogloss:0.16747
[13]	train-mlogloss:0.23711	eval-mlogloss:0.16726
[14]	train-mlogloss:0.23633	eval-mlogloss:0.16656
[15]	train-mlogloss:0.23624	eval-mlogloss:0.16641
[16]	train-mlogloss:0.23578	eval-mlogloss:0.16601
[17]	train-mlogloss:0.23535	eval-mlogloss:0.16605
[18]	train-mlogloss:0.23508	eval-mlogloss:0.16645
[19]	train-mlogloss:0.23514	eval-mlogloss:0.16672
[20]	train

[I 2024-11-25 11:24:36,083] Trial 1 finished with value: 1.0 and parameters: {'lambda': 3.959844694942787e-06, 'alpha': 0.021514107982753786, 'eta': 0.2953149375502616, 'gamma': 0.0019492018762115107, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.7337965674726503, 'colsample_bytree': 0.704134552387603}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.00516	eval-mlogloss:1.00075


[I 2024-11-25 11:24:49,313] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.05819	eval-mlogloss:1.05646
[1]	train-mlogloss:1.02393	eval-mlogloss:1.02204
[2]	train-mlogloss:0.98741	eval-mlogloss:0.98335
[3]	train-mlogloss:0.95335	eval-mlogloss:0.94892


[I 2024-11-25 11:24:49,422] Trial 49 pruned. Trial was pruned at iteration 4.


Best trial: {'lambda': 0.1862281922948573, 'alpha': 0.0002686829395108523, 'eta': 0.2010747597313131, 'gamma': 1.8671245320624107e-07, 'max_depth': 9, 'min_child_weight': 4, 'subsample': 0.7537517770115805, 'colsample_bytree': 0.9625969913585541}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()